In [1]:
from pathlib import Path
import torch
import pandas as pd
import stat
import numbers
import pydicom
import numpy as np
from tqdm import tqdm
from collections import Counter
import re
import os
from PIL import Image
from matplotlib import pyplot as plt
import shutil

In [2]:
Path().resolve()

PosixPath('/home/buehlern/Documents/Masterarbeit/notebooks')

In [3]:
df_loc = Path('cache/prep_df.pkl')
df = pd.read_pickle(df_loc)

In [4]:
def open_images(paths):
    for p in paths:
        image = pydicom.read_file(p).pixel_array

# Copy to local storage

In [5]:
paths = df.sample(1000)['path']

In [6]:
%%time
local_paths = []
for i, p in enumerate(paths):
    local_p = Path() / ('benchmark-cache/dicom/img-' + str(i))
    shutil.copyfile(p, local_p)
    local_paths.append(local_p)

CPU times: user 3.63 s, sys: 20.4 s, total: 24 s
Wall time: 3min 9s


# Open from NAS vs local

In [7]:
%%time
open_images(paths)

CPU times: user 10.8 s, sys: 13.7 s, total: 24.5 s
Wall time: 2min 50s


In [8]:
%%time
open_images(local_paths)

CPU times: user 2.82 s, sys: 3.08 s, total: 5.91 s
Wall time: 6.07 s


# Saving as .npy

In [9]:
%%time
npy_paths = []
npy_paths_local = []
for i, p in enumerate(paths):
    npy_p = Path('/home/buehlern/neocortex-nas/buehlern/Masterarbeit') / ('benchmark-cache/npy/img-' + str(i) + '.npy')
    npy_p_l =  Path() / ('benchmark-cache/npy/img-' + str(i) + '.npy')
    image = pydicom.read_file(p).pixel_array
    np.save(npy_p, image)
    np.save(npy_p_l, image)
    npy_paths.append(npy_p)
    npy_paths_local.append(npy_p_l)

CPU times: user 15.5 s, sys: 35 s, total: 50.5 s
Wall time: 7min 10s


In [10]:
def open_images_npy(paths):
    for p in paths:
        image = np.load(p)

In [11]:
%%time
open_images_npy(npy_paths)

CPU times: user 3.55 s, sys: 12.7 s, total: 16.2 s
Wall time: 2min 25s


In [12]:
%%time
open_images_npy(npy_paths_local)

CPU times: user 252 ms, sys: 3.43 s, total: 3.68 s
Wall time: 3.88 s
